In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from skimage import io, transform

import pandas as pd 
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from matplotlib.pyplot import figure
import os, time
from scipy.ndimage.filters import gaussian_filter1d
from datetime import date, datetime
from random import randrange
import math
import time
from datetime import datetime
import pytz
from tqdm import tqdm
from sklearn.model_selection import KFold


import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import ipywidgets as widgets

## Selecting Dataset

In [ ]:
DatasetName = widgets.Dropdown(options=['July', 'October', 'Both'], value='October', description='Number:', disabled=False)
display(DatasetName)

In [ ]:
JulyDataset, OctoberDataset, All = 0, 0, 0

print('We are using Dataset: ', DatasetName.value)

if DatasetName.value == 'July':
    JulyDataset = 1
if DatasetName.value == 'October':
    OctoberDataset = 1
if DatasetName.value == 'Both':
    All = 1

In [ ]:
if JulyDataset==1:
#     NikPath = "H:/HDD1/Field Test 7-29-21/Nikon Photos/ManualAnnotated/Patch Size/Size_ 256_256/**/*.JPG"
    NikPath = "H:/HDD1/Field Test 7-29-21/Nikon Photos/ManualAnnotated/ROI with Manual and VidStab/**/*.JPG"
    cn2path = ["H:/HDD1/Field Test 7-29-21/Scint Logs/20210729_180021.log"] 
    distance = 724
    
    cndfList = [pd.read_csv(a, sep=r'\t', engine='python', encoding= 'unicode_escape') for a in cn2path]
    cndf = pd.concat([a.iloc[1:] for a in cndfList])
    cndf = cndf.iloc[128:296]#.iloc[::2]
    cndf.drop(cndf.index[cndf['Cn2'] == '0.000000'], inplace=True)

if OctoberDataset==1:
#     NikPath = "H:/HDD1/Crop/Field Test 10-28-21-object-multi-MotionW-WO/WithPlatformCorrected/Nikon Photos/**/*.JPG"
    NikPath = "E:/Research/Alphacore/Cn2 Image Dataset/October Manual and Vidstab ROI/**/*.JPG"
    cn2path = ["H:/HDD1/Field Test 10-28-21/Scintillometer/20211028_180021.log","H:/HDD1/Field Test 10-28-21/Scintillometer/20211029_180021.log"]
    distance = 1450 

    
    cndfList = [pd.read_csv(a, sep=r'\t', engine='python', encoding= 'unicode_escape') for a in cn2path]#[1]
    cndf = pd.concat([a.iloc[1:] for a in cndfList])
    cndf = cndf.iloc[1206:].iloc[::2]
    
    
if All==1:
    NikPath = ["H:/HDD1/Field Test 7-29-21/Nikon Photos/ManualAnnotated/ROI with Manual and VidStab/**/*.JPG", "H:/HDD1/Crop/Field Test 10-28-21-Manual and vidStab_ROI/Nikon Photos/**/*.JPG"]
    cn2path = ["H:/HDD1/Field Test 10-28-21/Scintillometer/20211028_180021.log","H:/HDD1/Field Test 10-28-21/Scintillometer/20211029_180021.log"]
    
    cndfList = [pd.read_csv(a, sep=r'\t', engine='python', encoding= 'unicode_escape') for a in cn2path]#[1]
    cndf = pd.concat([a.iloc[1:] for a in cndfList])
    cndfOct = cndf.iloc[1206:].iloc[::2]
    
    cn2path = ["H:/HDD1/Field Test 7-29-21/Scint Logs/20210729_180021.log"] 
    cndfList = [pd.read_csv(a, sep=r'\t', engine='python', encoding= 'unicode_escape') for a in cn2path]
    cndf = pd.concat([a.iloc[1:] for a in cndfList])
    cndf = cndf.iloc[128:296]#.iloc[::2]
    cndf.drop(cndf.index[cndf['Cn2'] == '0.000000'], inplace=True)
    cndf = pd.concat([cndf, cndfOct], ignore_index=True, sort=False)

## Combining all Images and Scintillometer Data Together in CSV file

In [ ]:
# Correcting the Date-Time on Scintillometer Data
for i in range(len(cndf)):
    sciPreviusTimstamp = datetime.strptime(cndf['DateTime'].iloc[i], "%Y/%m/%d %H:%M:%S").timestamp()-7*3600  # The existed Data was 7 Hours added
    cndf['DateTime'].iloc[i] = datetime.fromtimestamp(sciPreviusTimstamp)#.astimezone(pytz.timezone('GMT'))   # So, We don't really need to convert to timezone
    
Cn2Max, Cn2Min = cndf['Cn2'].astype(float).max(),  cndf['Cn2'].astype(float).min()
def Cn2deNorm(cn2):
    return cn2*(Cn2Max-Cn2Min)+Cn2Min

cndf['Cn2_Normalized'] = cndf['Cn2'].apply(lambda t: (float(t)-Cn2Min)/(Cn2Max - Cn2Min))

NumOfCn2 = len(cndf)
display(cndf)

In [ ]:
NikDf = pd.DataFrame({'Nikon Time': [], 'Nikon Path' : [], 'Nikon TimeStamp': []})

if All==1:
    imageList = glob.glob(NikPath[0]) + glob.glob(NikPath[1])
else:
    imageList = glob.glob(NikPath)
    
for i, name in enumerate(tqdm(imageList)):
    tStamp = Image.open(name)._getexif()[36867]
    thatTime = datetime.strptime(tStamp, '%Y:%m:%d %H:%M:%S')

    NikDf1 = pd.DataFrame({'Nikon Time': [thatTime], 'Nikon Path' : [name], 'Nikon TimeStamp': [tStamp]})
    NikDf = NikDf.append(NikDf1, ignore_index = True)
    
# Replace All Second values to Zeros, And based on that Change the TimeStamp
NikDf['Nikon Time'] = pd.to_datetime(NikDf['Nikon Time'])
NikDf['Nikon Time'] = NikDf['Nikon Time'].apply(lambda t: t.replace(second=0))
NikDf['Nikon TimeStamp'] = NikDf['Nikon Time'].apply(lambda t: t.timestamp())
NikDf['Nikon Crop'] = NikDf['Nikon Path'].apply(lambda t: t.replace("\\", '/'))
display(NikDf)

# Join Both Scintillometer and Nikon Image path into Same CSV File
NikSinDF = NikDf.join(cndf.set_index('DateTime'), on='Nikon Time', how='inner')
NikSinDF.reset_index(drop=True, inplace=True)
NikSinDF.to_csv(f'{DatasetName.value}_Combined.csv')
display(NikSinDF)

In [ ]:
# NikSinDF.iloc[26220]['Nikon Path']#.split('/')[2]

October Dataset Start from index **26220**

In [ ]:
plt.rcParams['figure.figsize'] = (20,12)
plt.yscale('log',base=10), plt.grid(alpha=0.1)
plt.plot(np.array(NikSinDF['Cn2'][:26219].astype(float))), plt.title('Cn2 Graph in Log Scale', size=20);

# Dataset Class

In [ ]:
from torch.utils.data import Dataset, DataLoader

class cn2Dataset(Dataset):
    
    def __init__(self, dataframe, transform=None):
        self.cubeCh = cubeChNum # 3
        self.features = dataframe[:self.cubeCh*math.floor(len(dataframe)/self.cubeCh)]
        self.root_dir = dataframe['Nikon Crop']
        self.transform = transform
        
        length = int(len(self.features)/self.cubeCh)
        self.data = np.zeros((length, self.cubeCh, 256, 256))

    def __len__(self):
        return int(len(self.features)/self.cubeCh)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        if self.data[idx,:,:].sum()==0:        
            Image20 = np.zeros((self.cubeCh,256,256))
            
            for j in range(self.cubeCh):
                img_name = self.root_dir[idx*self.cubeCh+j]
                image = io.imread(img_name)[:,:,0]
                Image20[j, :, :] = image

            assert len(Image20.shape) == 3 and Image20.shape[0] == self.cubeCh
            Image20 = Image20[:, :, :]/255.0
            self.data[idx,:, :,:] = Image20
        
        Image20 = self.data[idx,:, :,:]
        Image20 = torch.from_numpy(Image20)
        Image20 = Image20.float().to(device)

        cn2_features = self.features['Cn2_Normalized'][idx*self.cubeCh]#.iloc[idx*self.cubeCh, 2]
        cn2_features = torch.from_numpy(np.array(cn2_features, dtype=np.float32)).to(device)
        
#         if OctoberDataset==1:
#             cn2_features = cn2_features/2 # October dataset have Double the distance
            
#         if All ==1 and cubeChNum*idx>=26220:
#             cn2_features = cn2_features/2 # October dataset have Double the distance, and final output is Cn2/Lenght. It's relative
        
        if self.transform:
            Image20 = self.transform(Image20)
            
        sample = {'image': Image20, 'features': cn2_features}

#             if self.transform:
#                 sample['image'] = transform.resize(sample['image'],(64,64)).reshape(3,64,64).to(device)

#         sample['image'] = image# torch.from_numpy(sample['image'][:,:,:]).float32().to(device)
#         sample['features'] = cn2_features

        return sample

# Some Parameters To Choose

In [ ]:
lrate = 1e-4 #1e-4 best for 10x10 kernel
epochCount = 1000
batch_s = 1
cubeChNum = 3 #Number of Images
kernelSize = 10
AdditionalLayer = 0
GradientCh = 1
GraphPath = "E:/Research/Alphacore/Paper/Graph/Graph Data/Diff Gradient CNN/October/Trained on July tested on October/"
TestPortion = 2
Subsample = 1 # Select every nth Images
Opposit = 0  #  0 -> Train on July and test on october, 1-> Opposit
distanceFactor = 0.5 # In transfer model, 2 -> Train on July and Test on October, 0.5 -> viseversa 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Usual Dataloader

In [ ]:
# Add Training IDs and Test Ids Based on 180 Values
if DatasetName.value != 'Both':
    TotalDataset = cn2Dataset(dataframe = NikSinDF[::Subsample], transform=None)

    L = len(TotalDataset)
    train_ids = []
    valid_ids = []
    test_ids = []

    it = 0
    for a in range(L):
        if int(a*NumOfCn2/L) == it:  #NumOfCn2 = Number of Scintillometer Unique Values
            valid_ids.append(a)
            it = it+1
        elif a%TestPortion==0:  #How many portion to be used for Testing
            test_ids.append(a)
        else:
            train_ids.append(a)

    train_ids, valid_ids, test_ids = np.array(train_ids), np.array(valid_ids), np.array(test_ids)

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(TotalDataset, shuffle=False, batch_size=batch_s, sampler=train_subsampler)
    validloader = torch.utils.data.DataLoader(TotalDataset,shuffle=False, batch_size=batch_s, sampler=valid_subsampler)
    testloader = torch.utils.data.DataLoader(TotalDataset,shuffle=False, batch_size=batch_s, sampler=test_subsampler)

    print(f'Number of training Data = {len(train_ids)}, and Number of Test dataset = {len(test_ids)}')

# Extrapolation Data Loader

In [ ]:
# Add julying IDs and Test Ids Based on 180 Values
if DatasetName.value == 'Both':
    TotalDataset = cn2Dataset(dataframe = NikSinDF[::Subsample], transform=None)

    L = len(TotalDataset)
    july_ids = []
    october_ids = []

    it = 0
    for a in range(L):
        if cubeChNum*a<26220:  #NumOfCn2 = Number of Scintillometer Unique Values
            july_ids.append(a)
        else:
            october_ids.append(a)

    july_ids, october_ids = np.array(july_ids), np.array(october_ids)

    july_subsampler = torch.utils.data.SubsetRandomSampler(july_ids)
    october_subsampler = torch.utils.data.SubsetRandomSampler(october_ids)

    # Define data loaders for julying and testing data in this fold
    julyloader = torch.utils.data.DataLoader(TotalDataset, shuffle=False, batch_size=batch_s, sampler=july_subsampler)
    octoberloader = torch.utils.data.DataLoader(TotalDataset,shuffle=False, batch_size=batch_s, sampler=october_subsampler)

    print(f'Number of julying Data = {len(july_ids)}, and Number of october dataset = {len(october_ids)}')

## Show Video Sequence of a data Cube

In [ ]:
%matplotlib agg 
# To avoild display duplicate image beside video
def showSample(input_sample):
    numFrames = input_sample.shape[-1]
    fig = plt.figure()
    im = plt.imshow(input_sample[:,:,0], cmap='gray')

    def update(i):
        img = input_sample[:,:,i]
        im.set_data(img)
        return im

    ani = animation.FuncAnimation(fig, update, frames=numFrames, repeat=False)  
    display(HTML(ani.to_html5_video()))
    
print('Lenght of total Dataset = ',len(TotalDataset))

pos = 1700
aDataCube = TotalDataset[pos]['image'].cpu()
aDataCube = np.transpose(aDataCube, (1,2,0))
showSample(aDataCube)
%matplotlib inline

## Niave CNN Deep Learning

## Deep Learning Model (Net -> net)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

def GradientDiffeciable(img):
    Ix = torch.diff(img, n=1, dim=-1)
    Ix = F.pad(input=Ix, pad=(0, 1, 0, 0), mode='replicate', value=0)

    Iy = torch.diff(img, n=1, dim=-2)
    Iy = F.pad(input=Iy, pad=(0, 0, 0, 1), mode='replicate', value=0)

    Grad_img = Ix**2 + Iy**2    # It's weird but they use not square root
    Grad_img[Grad_img < Grad_img.max()/100] = torch.nan
    return Grad_img

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(cubeChNum,GradientCh,kernelSize, padding='same')
    self.conv2 = nn.Conv2d(GradientCh,GradientCh,kernelSize, padding='same')
    self.conv3 = nn.Conv2d(GradientCh,GradientCh,kernelSize, padding='same')
#     self.conv4 = nn.Conv2d(GradientCh,GradientCh,kernelSize, padding='same')

  def forward(self, x):
    kvar = torch.var(x.squeeze(), dim=0)
    
    x = F.relu(self.conv1(x))    
    x = F.relu(self.conv2(x))    
#     x = F.relu(self.conv3(x))     
#     x = F.relu(self.conv4(x))
#     for i in range(AdditionalLayer):
#         x = F.relu(self.conv2(x))
    kGradStack = F.relu(self.conv3(x))
    
    kGradStack = torch.abs(kGradStack)
    
    kGradAvg = torch.nanmean(kGradStack, dim=0)
    kGradAvg[kGradAvg < kGradAvg.max()/100] = torch.nan
    kdiv = 0.00001/(torch.nanmean(kvar*kGradAvg))
#     kdiv = 0.001/(torch.nanmean(kvar/kGradAvg))
    return kdiv#, kvar, kGradAvg, x

net = Net().to(device)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of Parameters = {count_parameters(net):,}')

## Load the Model if you wish

In [ ]:
# net.load_state_dict(torch.load('model/DiffGrad_O_AL0_abs_sin.pth'))
# plotTestResultSave(test_ids, f'{GraphName}, Epoch {epoch}')

## Optimizer

In [ ]:
criterion = nn.L1Loss()     #nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=lrate, momentum=0.9)
optimizer = torch.optim.Adam(net.parameters(), lr=lrate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

## Test Accuracy Measurement/ Graph

In [ ]:
# def AccuracyNow():
#     Otest, Ptest, AllError = [], [], []
#     for i, data in (enumerate(testloader, 0)):
#         inputs, labels = data['image'], data['features']
#         output = net(inputs)

#         output = output.cpu().detach().numpy().squeeze()
#         labels = labels.cpu().detach().numpy().squeeze()
#         BatchError = np.absolute(output-labels).mean()
#         AllError.append(BatchError)

#         Otest.append(labels)
#         Ptest.append(output)

#     Error = np.array(AllError).mean()
#     return Error, Otest, Ptest

def saveResult(ori, pre, name):
    TestError = np.absolute(np.array(ori)-np.array(pre)).mean()
    TestError = str(np.round(TestError,18))
    #print(TestError)
    
    current_time = datetime.now().strftime("%D__%H-%M-%S")
    cTimeStr = str(current_time).replace('/','.')
    #print(cTimeStr)

    TestTable = {'Original': ori, 'Predicted': pre}
    testTableDf = pd.DataFrame(TestTable)
    testTableDf.to_csv(f"{GraphPath}{name}{cTimeStr}.csv")
    #print(f'Data saved to: {GraphPath}{name}{cTimeStr}.csv')
    
    plt.rcParams['figure.figsize'] = [20, 8]    
    plt.yscale('log',base=10), plt.grid(alpha=0.1)
    plt.plot(ori, label = "Original")
    plt.plot(pre, label = "Predicted")
    plt.title(f'{name} Error: {TestError}.png', size=20)
    plt.legend(), plt.grid()
    plt.savefig(f"{GraphPath}{name}{cTimeStr}.png")
    # print(f'Figure saved to: {GraphPath}/{name}{cTimeStr}.csv')
    plt.show()
    
def plotTestResultSave(DatasetIds, name):
    ind = []
    original = []
    predicted = []
    inc = 0

    for i in tqdm(DatasetIds, desc='Test '):
        inputs, labels = TotalDataset[i]['image'], TotalDataset[i]['features']
        
        
        # Multiplying with Distance factor of 2 when traingin on October and testing on July
        
        output = net(inputs.reshape(1,cubeChNum,256,256)).squeeze()*distanceFactor
        ind.append(i)
        original.append(Cn2deNorm(labels).cpu().detach().numpy().squeeze())
        predicted.append(Cn2deNorm(output).cpu().detach().numpy().squeeze())
        inc+=1
    saveResult(original, predicted, name)

## Training Interpolation

In [ ]:
# %%script false --no-raise-error
if DatasetName.value != 'Both':
    GraphName = f'DiffGrad_{DatasetName.value}'
    LossArray = []
    TestLossArray = []

    for epoch in range(epochCount):    
        net.train()
        running_loss = 0

        pbar = tqdm(trainloader, desc="Train")
        for i, data in (enumerate(pbar, 0)):
            optimizer.zero_grad()

            inputs, labels = data['image'], data['features']
            output = net(inputs).squeeze()
#             loss = torch.mean(torch.absolute(labels-output))
            loss = torch.mean(torch.absolute(labels-output)/torch.absolute(labels+output))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i%100 == 99:      
                pbar.set_postfix({'Loss100':  np.round(running_loss, 3)})
                running_loss = 0.0

        net.eval()
        plotTestResultSave(test_ids, f'GraphName, Epoch {epoch}')
    #     Err, _, _ = AccuracyNow()
    #     TestLossArray.append(Err)
    #     print(f'Epoch {epoch}, Test Error = {str(np.round(Cn2deNorm(Err),18))}, {str(np.round(Err,6))}')
    print('Finished Training')

In [ ]:
# torch.save(net.state_dict(), 'model/DiffGrad_O_AL0_abs_sin.pth')

# Training Extrapolaiton Model Transfer

In [ ]:
if DatasetName.value == 'Both':
    GraphName = f'DiffGrad_{DatasetName.value}'
    LossArray = []
    TestLossArray = []

    for epoch in range(epochCount):    
        net.train()
        running_loss = 0

        
        if Opposit==0:
            pbar = tqdm(julyloader, desc="Train")
        else:
            pbar = tqdm(octoberloader, desc="Train") 
        
        
        for i, data in (enumerate(pbar, 0)):
            optimizer.zero_grad()

            inputs, labels = data['image'], data['features']
            output = net(inputs).squeeze()
            loss = torch.mean(torch.absolute(labels-output))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i%100 == 99:      
                pbar.set_postfix({'Loss100':  np.round(running_loss, 3)})
                running_loss = 0.0

        net.eval()
        if Opposit==0:
            plotTestResultSave(october_ids, f'{GraphName}, Epoch {epoch}')
        else:
            plotTestResultSave(july_ids, f'{GraphName}, Epoch {epoch}')
            
    #     Err, _, _ = AccuracyNow()
    #     TestLossArray.append(Err)
    #     print(f'Epoch {epoch}, Test Error = {str(np.round(Cn2deNorm(Err),18))}, {str(np.round(Err,6))}')
    print('Finished Training')

In [ ]:
break

## Extrapolation

In [ ]:
# TimeFold True = Extrapolation, False = Intrapolation
Extrapolation = True

In [ ]:
def reset_weights(m):
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

# FoldNumber = [10, 8, 6, 5, 4, 3, 2]
FoldNumber = [6]


for FN in FoldNumber:
    kfold = KFold(n_splits=FN, shuffle=not Extrapolation)

    FoldErrors = []
    FoldPErrors = []

    # K-fold Cross Validation model evaluation
    for fold, (train_ids, test_ids) in enumerate(kfold.split(TotalDataset)):
        net.train()

        # net = Net().to(device)
        net = effnetv2_s().to(device) 
        net.apply(reset_weights)
        #optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
        optimizer = torch.optim.Adam(net.parameters(), lr=lrate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
        #optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)


        # print(test_ids)
        ##############################   K-Fold Dataloader    ##########################################
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

        # Define data loaders for training and testing data in this fold
        trainloader = torch.utils.data.DataLoader(TotalDataset, batch_size=batch_s, sampler=train_subsampler)
        testloader = torch.utils.data.DataLoader(TotalDataset, batch_size=batch_s, sampler=test_subsampler)

        ##############################   Training The Net    ###########################################
        print(f"{'#'*70}\n \t\t\t Training Fold: {fold}\n{'#'*70}")

        EpochCount = 4
        LossArray = []
        # print('epoch', '\t', 'Dataset Loss', '\t', 'Pixel difference')

        for epoch in range(EpochCount):
            running_loss = 0
            for i, data in (enumerate(trainloader, 0)):
                optimizer.zero_grad()

                inputs, labels = data['image'], data['features']
                output = net(inputs).squeeze()

                # print(labels, output)
                #loss = criterion(output, labels)
                loss = torch.mean(torch.absolute(labels-output))

                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(epoch, '->',  np.round(running_loss, 3), end='\t')

        print('Finished Training')

        ###############################   Evaluating The model #########################################
        net.eval()
#         ErrorThisFold, _, _ = AccuracyNow()
#         FoldErrors.append(ErrorThisFold)
    #     FoldPErrors.append(Perror)
        plotTestResultSave(test_ids, f'Extrapolation_{FoldNumber}Epoch FN {FN} - Fold {fold} ')

    FoldErrors = np.array([float(a) for a in FoldErrors])
    FoldPErrors = np.array([float(a) for a in FoldPErrors])
    print("Total Error = ", FoldErrors.mean())
    print("Percentage Error = ", FoldPErrors.mean())

